In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score, confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/AILAB/final_dataset.csv')
print(df.shape)
df.head()

(2669, 6)


,id,text,Garge Archana Atul,Archana Kumari,Priyanshu Raj,Final
0,1,Sassy...? More like femme bots than killer blo...,1,1.0,1,1.0
1,2,I've had better looking shits than these two! ...,1,1.0,2,1.0
2,3,The girls can cook for me anytime. Just not ho...,1,1.0,2,1.0
3,4,The face of very ugly promo girls ! Faces like...,1,1.0,1,1.0
4,5,@mykitchenrules Elegant and beautiful?Cheap an...,1,1.0,1,1.0


In [ ]:
df.isna().sum()

id                    0
text                  0
Garge Archana Atul    0
Archana Kumari        1
Priyanshu Raj         0
Final                 5
dtype: int64

In [ ]:
df=df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)

In [ ]:
df['Final'].value_counts()

1.0    1539
2.0    1041
0.0      83
Name: Final, dtype: int64

In [ ]:
%%time
#text preprocessing
#data cleaning and preprocessing
import re

text=' '.join(df['text'])
text=text.split()
freq_comm = pd.Series(text).value_counts()
rare=freq_comm[freq_comm.values==1]

def get_clean_text(x):
  if type(x) is str:
    x=x.lower()
    x=re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x)
    x=re.sub('rt', '', x)
    x=re.sub('[^A-Z a-z]+','',x)
    x=' '.join([t for t in x.split() if t not in rare])
    return x
  else:
    return x
df['text']=df['text'].apply(lambda x: get_clean_text(x))


CPU times: user 103 ms, sys: 2.58 ms, total: 105 ms
Wall time: 104 ms


In [ ]:
y=df['Final'].to_numpy()
print(y)
print(y.shape)

[1. 1. 1. ... 2. 2. 2.]
(2663,)


**tf-idf**

In [ ]:
#tf-idf method of vectorization
Tfidf_vect = TfidfVectorizer(max_features=10000)
X=Tfidf_vect.fit_transform(df['text']).toarray()
print(X)
print(X.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2663, 5845)


In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.30, random_state=0)

**tf-idf char ngram**

In [ ]:
Tfidf_vect = TfidfVectorizer(analyzer='char', ngram_range=(2,5),max_features=10000)
X_char_ngram=Tfidf_vect.fit_transform(df['text']).toarray()
print(X_char_ngram)
print(X_char_ngram.shape)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.02302567 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.03285456 0.02701675 0.         ... 0.         0.         0.        ]
 [0.01850257 0.         0.         ... 0.         0.         0.        ]
 [0.05638252 0.0309094  0.         ... 0.         0.         0.        ]]
(2663, 10000)


In [ ]:
#split for character n-gram vectorization
X_train_char, X_test_char, y_train_char, y_test_char=train_test_split(X_char_ngram,y, test_size=0.30, random_state=0)

**tf-idf with word n-gram**

In [ ]:
Tfidf_vect_word = TfidfVectorizer(ngram_range=(2,5),max_features=10000)
X_word_ngram=Tfidf_vect.fit_transform(df['text']).toarray()
print(X_word_ngram)
print(X_word_ngram.shape)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.02302567 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.03285456 0.02701675 0.         ... 0.         0.         0.        ]
 [0.01850257 0.         0.         ... 0.         0.         0.        ]
 [0.05638252 0.0309094  0.         ... 0.         0.         0.        ]]
(2663, 10000)


In [ ]:
#split for word n-gram vectorization
X_train_word, X_test_word, y_train_word, y_test_word=train_test_split(X_word_ngram,y, test_size=0.30, random_state=0)

##**Gradient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.25, max_features=2, max_depth=2, random_state=0)
gb_clf = GradientBoostingClassifier()
model1=gb_clf.fit(X_train, y_train)

print("Score for training data : {0:.3f}".format(model1.score(X_train, y_train)))
print("Score for testing data : {0:.3f}".format(model1.score(X_test, y_test)))

y_pred=model1.predict(X_test)
y_pred_train=model1.predict(X_train)

#calculate accuracy
print("Accuracy Score of gradient boosting Model on test data : ",accuracy_score(y_test, y_pred))
print("Accuracy Score of gradient boosting Model on training data : ",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Score for training data : 0.876
Score for testing data : 0.786
Accuracy Score of gradient boosting Model on test data :  0.785982478097622
Accuracy Score of gradient boosting Model on training data :  0.8755364806866953
              precision    recall  f1-score   support

         0.0       0.29      0.09      0.14        22
         1.0       0.88      0.77      0.82       462
         2.0       0.70      0.85      0.77       315

    accuracy                           0.79       799
   macro avg       0.62      0.57      0.58       799
weighted avg       0.79      0.79      0.78       799

Confusion Matrix:
[[  2   6  14]
 [  1 358 103]
 [  4  43 268]]


In [ ]:
#for word n gram
model2=gb_clf.fit(X_train_word, y_train_word)
print("Score for training data : {0:.3f}".format(model2.score(X_train_word, y_train_word)))
print("Score for testing data : {0:.3f}".format(model2.score(X_test_word, y_test_word)))
y_pred_gword=model2.predict(X_test_word)
y_pred_train_gword=model2.predict(X_train_word)
# for word n gram 
print(classification_report(y_test_word,y_pred_gword))
print("Confusion Matrix:")
print(confusion_matrix(y_test_word, y_pred_gword))

Score for training data : 0.931
Score for testing data : 0.790
              precision    recall  f1-score   support

         0.0       0.29      0.09      0.14        22
         1.0       0.86      0.81      0.83       462
         2.0       0.72      0.82      0.76       315

    accuracy                           0.79       799
   macro avg       0.62      0.57      0.58       799
weighted avg       0.79      0.79      0.78       799

Confusion Matrix:
[[  2   7  13]
 [  3 372  87]
 [  2  56 257]]


In [ ]:
#for char n gram
model3=gb_clf.fit(X_train_char, y_train_char)
print("Score for training data : {0:.3f}".format(model3.score(X_train_char, y_train_char)))
print("Score for testing data : {0:.3f}".format(model3.score(X_test_char, y_test_char)))
y_pred_gchar=model3.predict(X_test_char)
y_pred_train_gchar=model3.predict(X_train_char)
# for char n gram 
print(classification_report(y_test_char,y_pred_gchar))
print("Confusion Matrix:")
print(confusion_matrix(y_test_char, y_pred_gchar))

Score for training data : 0.932
Score for testing data : 0.781
              precision    recall  f1-score   support

         0.0       0.29      0.09      0.14        22
         1.0       0.85      0.79      0.82       462
         2.0       0.71      0.81      0.75       315

    accuracy                           0.78       799
   macro avg       0.61      0.56      0.57       799
weighted avg       0.78      0.78      0.78       799

Confusion Matrix:
[[  2   6  14]
 [  3 367  92]
 [  2  58 255]]


##**AdaBoosting**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
#clf = AdaBoostClassifier(n_estimators=195, random_state=0)
clf=AdaBoostClassifier()
model4=clf.fit(X_train, y_train)
y_pred_adaboost=model4.predict(X_test)
y_pred_train_adaboost=model4.predict(X_train)
print("Score for test data : ", model4.score(X_test, y_pred))
print("Score for training data : " ,model4.score(X_train, y_pred_train))
print(classification_report(y_test,y_pred_adaboost))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Score for test data :  0.8811013767209012
Score for training data :  0.8476394849785408
              precision    recall  f1-score   support

         0.0       0.20      0.09      0.13        22
         1.0       0.86      0.69      0.76       462
         2.0       0.63      0.83      0.72       315

    accuracy                           0.73       799
   macro avg       0.56      0.54      0.54       799
weighted avg       0.75      0.73      0.73       799

Confusion Matrix:
[[  2   6  14]
 [  1 358 103]
 [  4  43 268]]


In [ ]:
#for word n gram
model5=clf.fit(X_train_word, y_train_word)
print("Score for training data : {0:.3f}".format(model5.score(X_train_word, y_train_word)))
print("Score for testing data : {0:.3f}".format(model5.score(X_test_word, y_test_word)))
y_pred_word_adaboost=model5.predict(X_test_word)
y_pred_train_word_adaboost=model5.predict(X_train_word)
#char word n gram adaboosting
print(classification_report(y_test_word,y_pred_word_adaboost))
print("Confusion Matrix:")
print(confusion_matrix(y_test_word, y_pred_word_adaboost))

Score for training data : 0.745
Score for testing data : 0.725
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.81      0.73      0.77       462
         2.0       0.63      0.77      0.70       315

    accuracy                           0.72       799
   macro avg       0.48      0.50      0.49       799
weighted avg       0.72      0.72      0.72       799

Confusion Matrix:
[[  0   8  14]
 [  0 336 126]
 [  0  72 243]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#for char n gram
model6=clf.fit(X_train_char, y_train_char)
print("Score for training data : {0:.3f}".format(model6.score(X_train_char, y_train_char)))
print("Score for testing data : {0:.3f}".format(model6.score(X_test_char, y_test_char)))
y_pred_char_adaboost=model6.predict(X_test_char)
y_pred_train_char_adaboost=model6.predict(X_train_char)
#char n gram adaboosting
print(classification_report(y_test_char,y_pred_char_adaboost))
print("Confusion Matrix:")
print(confusion_matrix(y_test_char, y_pred_char_adaboost))

Score for training data : 0.745
Score for testing data : 0.723
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.81      0.73      0.76       462
         2.0       0.63      0.77      0.69       315

    accuracy                           0.72       799
   macro avg       0.48      0.50      0.49       799
weighted avg       0.72      0.72      0.72       799

Confusion Matrix:
[[  0   8  14]
 [  0 336 126]
 [  0  73 242]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##**XGD Boosting**

In [ ]:
from xgboost import XGBClassifier
#xgbc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,n_estimators=150, n_jobs=1, nthread=None,objective='multi:softprob', random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=None, subsample=1, verbosity=1) 
xgbc=XGBClassifier()
model7=xgbc.fit(X_train, y_train)
y_pred_xgbc=model7.predict(X_test)
y_pred_train_xgbc=model7.predict(X_train)
print("Score for training data : {0:.3f}".format(model7.score(X_train, y_train)))
print("Score for testing data : {0:.3f}".format(model7.score(X_test, y_test)))
print(classification_report(y_test,y_pred_xgbc))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgbc))

Score for training data : 0.818
Score for testing data : 0.777
              precision    recall  f1-score   support

         0.0       0.40      0.09      0.15        22
         1.0       0.87      0.76      0.81       462
         2.0       0.69      0.85      0.76       315

    accuracy                           0.78       799
   macro avg       0.65      0.57      0.57       799
weighted avg       0.78      0.78      0.77       799

Confusion Matrix:
[[  2   6  14]
 [  2 352 108]
 [  1  47 267]]


In [ ]:
model8=xgbc.fit(X_train_word, y_train_word)
y_pred_xgbc_word=model8.predict(X_test_word)
y_pred_train_xgbc_word=model8.predict(X_train_word)
print("Score for training data : {0:.3f}".format(model8.score(X_train_word, y_train_word)))
print("Score for testing data : {0:.3f}".format(model8.score(X_test_word, y_test_word)))
# USING WORD N GRAM 
print(classification_report(y_test_word,y_pred_xgbc_word))
print("Confusion Matrix:")
print(confusion_matrix(y_test_word, y_pred_xgbc_word))

Score for training data : 0.895
Score for testing data : 0.790
              precision    recall  f1-score   support

         0.0       0.67      0.09      0.16        22
         1.0       0.86      0.80      0.83       462
         2.0       0.71      0.83      0.76       315

    accuracy                           0.79       799
   macro avg       0.75      0.57      0.58       799
weighted avg       0.80      0.79      0.78       799

Confusion Matrix:
[[  2   7  13]
 [  0 368  94]
 [  1  53 261]]


In [ ]:
model9=xgbc.fit(X_train_char, y_train_char)
y_pred_xgbc_char=model9.predict(X_test_char)
y_pred_train_xgbc_char=model9.predict(X_train_char)
print("Score for training data : {0:.3f}".format(model9.score(X_train_char, y_train_char)))
print("Score for testing data : {0:.3f}".format(model9.score(X_test_char, y_test_char)))
# USING CHAR N GRAM 
print(classification_report(y_test_char,y_pred_xgbc_char))
print("Confusion Matrix:")
print(confusion_matrix(y_test_char, y_pred_xgbc_char))

Score for training data : 0.895
Score for testing data : 0.790
              precision    recall  f1-score   support

         0.0       0.67      0.09      0.16        22
         1.0       0.86      0.80      0.83       462
         2.0       0.71      0.83      0.76       315

    accuracy                           0.79       799
   macro avg       0.75      0.57      0.58       799
weighted avg       0.80      0.79      0.78       799

Confusion Matrix:
[[  2   7  13]
 [  0 368  94]
 [  1  53 261]]


##SVM##

**1. Linear SVM**

In [ ]:
from sklearn.svm import SVC
svm_classifier =SVC(C=1.0, kernel='linear',  gamma='auto')
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)
y_pred_svm_train=svm_classifier.predict(X_train)
cm_svm=confusion_matrix(y_test, y_pred_svm)
print("Confusion matrix of SVM Model:\n",cm_svm)
print("Accuracy Score of SVM Model on test data : ",accuracy_score(y_test, y_pred_svm))
print("Accuracy Score of SVM Model on training data : ",accuracy_score(y_train, y_pred_svm_train))
print(classification_report(y_test,y_pred_svm))

Confusion matrix of SVM Model:
 [[  1   7  14]
 [  0 382  80]
 [  2  62 251]]
Accuracy Score of SVM Model on test data :  0.7934918648310388
Accuracy Score of SVM Model on training data :  0.927038626609442
              precision    recall  f1-score   support

         0.0       0.33      0.05      0.08        22
         1.0       0.85      0.83      0.84       462
         2.0       0.73      0.80      0.76       315

    accuracy                           0.79       799
   macro avg       0.64      0.56      0.56       799
weighted avg       0.79      0.79      0.79       799



In [ ]:
#char n gram
svm_classifier.fit(X_train_char, y_train_char)
y_pred_svm = svm_classifier.predict(X_test_char)
y_pred_svm_train=svm_classifier.predict(X_train_char)
cm_svm=confusion_matrix(y_test_char, y_pred_svm)
print("Confusion matrix of SVM Model:\n",cm_svm)
print("Accuracy Score of SVM Model on test data : ",accuracy_score(y_test_char, y_pred_svm))
print("Accuracy Score of SVM Model on training data : ",accuracy_score(y_train_char, y_pred_svm_train))
print(classification_report(y_test_char,y_pred_svm))

Confusion matrix of SVM Model:
 [[  2   8  12]
 [  1 384  77]
 [  2  70 243]]
Accuracy Score of SVM Model on test data :  0.7872340425531915
Accuracy Score of SVM Model on training data :  0.9050429184549357
              precision    recall  f1-score   support

         0.0       0.40      0.09      0.15        22
         1.0       0.83      0.83      0.83       462
         2.0       0.73      0.77      0.75       315

    accuracy                           0.79       799
   macro avg       0.65      0.56      0.58       799
weighted avg       0.78      0.79      0.78       799



In [ ]:
#word n gram
svm_classifier.fit(X_train_word, y_train_word)
y_pred_svm = svm_classifier.predict(X_test_word)
y_pred_svm_train=svm_classifier.predict(X_train_word)
cm_svm=confusion_matrix(y_test_word, y_pred_svm)
print("Confusion matrix of SVM Model:\n",cm_svm)
print("Accuracy Score of SVM Model on test data : ",accuracy_score(y_test_word, y_pred_svm))
print("Accuracy Score of SVM Model on training data : ",accuracy_score(y_train_word, y_pred_svm_train))
print(classification_report(y_test_word,y_pred_svm))

Confusion matrix of SVM Model:
 [[  2   8  12]
 [  1 384  77]
 [  2  70 243]]
Accuracy Score of SVM Model on test data :  0.7872340425531915
Accuracy Score of SVM Model on training data :  0.9050429184549357
              precision    recall  f1-score   support

         0.0       0.40      0.09      0.15        22
         1.0       0.83      0.83      0.83       462
         2.0       0.73      0.77      0.75       315

    accuracy                           0.79       799
   macro avg       0.65      0.56      0.58       799
weighted avg       0.78      0.79      0.78       799



**2. RBF SVM**

In [ ]:
svm_classifier_rbf =SVC(C=1,kernel='rbf', gamma=0.1)
svm_classifier_rbf.fit(X_train, y_train)
y_pred_svm_rbf = svm_classifier_rbf.predict(X_test)
y_pred_svm_rbf_train = svm_classifier_rbf.predict(X_train)
cm_svm_rbf=confusion_matrix(y_test, y_pred_svm_rbf)
print("Confusion matrix of SVM Model:\n",cm_svm_rbf)
print("Accuracy Score of SVM Model on test data : ",accuracy_score(y_test, y_pred_svm_rbf))
print("Accuracy Score of SVM Model on train data : ",accuracy_score(y_train, y_pred_svm_rbf_train))
print(classification_report(y_test,y_pred_svm_rbf))

Confusion matrix of SVM Model:
 [[  0   6  16]
 [  0 387  75]
 [  0  75 240]]
Accuracy Score of SVM Model on test data :  0.7847309136420526
Accuracy Score of SVM Model on train data :  0.8465665236051502
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.83      0.84      0.83       462
         2.0       0.73      0.76      0.74       315

    accuracy                           0.78       799
   macro avg       0.52      0.53      0.53       799
weighted avg       0.76      0.78      0.77       799



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3. Poly SVM**

In [ ]:
svm_classifier_poly =SVC(kernel='poly', degree=2)
svm_classifier_poly.fit(X_train, y_train)
y_pred_svm_poly = svm_classifier_poly.predict(X_test)
y_pred_svm_poly_train = svm_classifier_poly.predict(X_train)
cm_svm_poly=confusion_matrix(y_test, y_pred_svm_poly)
print("Confusion matrix of SVM Model:\n",cm_svm_poly)
print("Accuracy Score of SVM Model on test data : ",accuracy_score(y_test, y_pred_svm_poly))
print("Accuracy Score of SVM Model on train data : ",accuracy_score(y_train, y_pred_svm_poly_train))
print(classification_report(y_test,y_pred_svm_poly))

Confusion matrix of SVM Model:
 [[  0   9  13]
 [  0 399  63]
 [  0  96 219]]
Accuracy Score of SVM Model on test data :  0.7734668335419274
Accuracy Score of SVM Model on train data :  0.9935622317596566
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.79      0.86      0.83       462
         2.0       0.74      0.70      0.72       315

    accuracy                           0.77       799
   macro avg       0.51      0.52      0.51       799
weighted avg       0.75      0.77      0.76       799



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##LOGISTIC REGRESSION##

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf=clf.fit(X_train, y_train)
y_pred_logistic=clf.predict(X_test)
y_pred_train_logistic=clf.predict(X_train)
cm_logistic=confusion_matrix(y_test, y_pred_logistic)
print("Confusion matrix of Logistic Regression Model:\n",cm_logistic)
print("Accuracy Score of logistic Model on test data : ",accuracy_score(y_test, y_pred_logistic))
print("Accuracy Score of logistic Model on training data : ",accuracy_score(y_train, y_pred_train_logistic))
print(classification_report(y_test,y_pred_logistic))

Confusion matrix of Logistic Regression Model:
 [[  0   7  15]
 [  0 387  75]
 [  0  71 244]]
Accuracy Score of logistic Model on test data :  0.7897371714643304
Accuracy Score of logistic Model on training data :  0.907725321888412
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.83      0.84      0.83       462
         2.0       0.73      0.77      0.75       315

    accuracy                           0.79       799
   macro avg       0.52      0.54      0.53       799
weighted avg       0.77      0.79      0.78       799



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Naive Bayes##

In [ ]:
#training using Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
tweet_analyse_model=MultinomialNB().fit(X_train, y_train)
y_pred_naive=tweet_analyse_model.predict(X_test)
y_pred_train_naive=twwet_analyse_model.predict(X_train)

print("Accuracy Score of logistic Model on test data : ",accuracy_score(y_test, y_pred_naive))
print("Accuracy Score of logistic Model on training data : ",accuracy_score(y_train, y_pred_train_naive))

#confusion matrix and report
print("Classification Report",classification_report(y_test,y_pred_naive))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_naive))

NameError: ignored